In [1]:
import json
import os
import pandas as pd
from difflib import SequenceMatcher
import jellyfish
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.distance import great_circle
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [2]:
#Center coordinates
SF_COORDINATES = (40.4168, -3.7038)

# for speed purposes
MAX_RECORDS = 10

In [3]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/data/')
os.listdir()

['.DS_Store',
 '20170614_BANCOS.csv',
 '20170614_PAPELERAS.csv',
 '20171110_InventarioFuentes.csv',
 'anexo i monumentos.xls',
 'artwork.geojson',
 'bench.geojson',
 'drinking_water.geojson',
 'indice.csv',
 'memorial.geojson',
 'monument.geojson',
 'street_lamp.geojson',
 'waste_basket.geojson']

In [4]:
OSM_fichero = 'street_lamp.geojson'
#AYU_fichero = '20171110_InventarioFuentes.csv'

In [5]:
with open(OSM_fichero) as f:
    OSM = json.load(f)
f.close()

In [6]:
ids = [OSM['features'][i]['id'] for i in range(len(OSM['features']))]
lat = [OSM['features'][i]['geometry']['coordinates'][1] for i in range(len(OSM['features']))]
lon = [OSM['features'][i]['geometry']['coordinates'][0] for i in range(len(OSM['features']))]
#name = [OSM['features'][i]['properties']['name'] for i in range(len(OSM['features']))]
ayu = ids[:]

In [7]:
lats = [lat[ids.index('node/2800720086')],lat[ids.index('node/2800668850')],lat[ids.index('node/2764886703')]]
longs = [lon[ids.index('node/2800720086')],lon[ids.index('node/2800668850')],lon[ids.index('node/2764886703')]]

points = list(zip(lats,longs))
points

[(40.4388632, -3.6537501), (40.438863, -3.6539007), (40.4399129, -3.6558072)]

In [8]:
def getDistance_meters(x,y):
    try:
        return great_circle(x, y).kilometers*1000.0
    except:
        return 0

In [9]:
getDistance_meters(points[0], points[1])

12.748932631098425

In [10]:
AYU_MISSING = []
for i in range(len(ids[:100])):
    if isinstance(ayu[i], str):
        AYU_MISSING.append((ids[i],lat[i],lon[i]))
        
AYU_MISSING = pd.DataFrame(AYU_MISSING)
AYU_MISSING.columns = ['id_OSM','LATITUD','LONGITUD']

AYU_MISSING = AYU_MISSING.set_index('id_OSM')
AYU_MISSING['coord'] = list(zip(AYU_MISSING.LATITUD.values, AYU_MISSING.LONGITUD.values))

In [11]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/csv/')
AYU_MISSING.to_csv(OSM_fichero[:-8]+'-missing.csv')

In [12]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/web/')

In [20]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11,max_zoom=21, control_scale = True, prefer_canvas=True)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_MISSING.index.values)):
    folium.Marker(icon=folium.Icon(color='green'),popup=folium.Popup(AYU_MISSING.index.values[i], parse_html=True),
        location = (AYU_MISSING.LATITUD.values[i],AYU_MISSING.LONGITUD.values[i])).add_to(marker_cluster)
map.save(OSM_fichero[:-8]+'-missing_Ayu.html')

In [33]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11,max_zoom=21)
HeatMap(AYU_MISSING['coord'].values,radius=50,max_zoom=21).add_to(map)
map.save(OSM_fichero[:-8]+'-missing_Ayu_heat.html')

In [ ]:
map